### SQL in 10 Minutes
#### Lesson VI
`subquery`, `join`, `rename`

In [1]:
import sqlite3
import pandas as pd

In [2]:
db = 'tysql.sqlite'
conn = sqlite3.connect(db)

In [3]:
tables = pd.read_sql("""SELECT * FROM sqlite_master WHERE type = 'table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,Customers,Customers,2,CREATE TABLE Customers\n(\n cust_id char...
1,table,OrderItems,OrderItems,4,CREATE TABLE OrderItems\n(\n order_num int ...
2,table,Orders,Orders,7,CREATE TABLE Orders\n(\n order_num int ...
3,table,Products,Products,10,CREATE TABLE Products\n(\n prod_id char(10...
4,table,Vendors,Vendors,12,CREATE TABLE Vendors\n(\n vend_id char(1...


Get the customers' id of the orders from product 'RGAN01'

In [4]:
# using subquery
sql = """
        select cust_id
        from Orders
        where order_num in (select order_num 
                                      from OrderItems
                                      where prod_id = 'RGAN01')
        """

In [5]:
ex1 = pd.read_sql(sql, conn)
ex1

,cust_id
0,1000000004
1,1000000005


In [6]:
# or simply using join to get the same query
sql = """
        select cust_id
        from Orders
        join Orderitems on Orders.order_num = Orderitems.order_num
        where OrderItems.prod_id = 'RGAN01'
        """

In [7]:
ex2 = pd.read_sql(sql, conn)
ex2

,cust_id
0,1000000004
1,1000000005


Keep going

In [8]:
sql = """
        select *
        from Customers;
        """

In [9]:
cu = pd.read_sql(sql, conn)
cu

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


In [10]:
sql = """
        select cust_name, cust_contact
        from Customers
        where cust_id in ('1000000004', '1000000005');
        """

In [11]:
ex3 = pd.read_sql(sql, conn)
ex3

,cust_name,cust_contact
0,Fun4All,Denise L. Stephens
1,The Toy Store,Kim Howard


Using subquery

In [12]:
sql = """
        select cust_name, cust_contact
        from Customers
        where cust_id in (select cust_id
                                 from Orders
                                 where order_num in (select order_num
                                                               from OrderItems
                                                               where prod_id = 'RGAN01'));
        """

In [13]:
ex4 = pd.read_sql(sql, conn)
ex4

,cust_name,cust_contact
0,Fun4All,Denise L. Stephens
1,The Toy Store,Kim Howard


In [14]:
sql = """
        select count(1) as orders
        from Orders
        where cust_id = '1000000001'
        """

In [15]:
ex5 = pd.read_sql(sql, conn)
ex5

,orders
0,2


Using subquery

In [22]:
sql = """
        select cust_name, cust_state, (select count(1)
                                                    from Orders
                                                    where Orders.cust_id = Customers.cust_id) as orders
        from Customers
        order by cust_name;
        """

In [23]:
ex6 = pd.read_sql(sql, conn)
ex6

,cust_name,cust_state,orders
0,Fun4All,IN,1
1,Fun4All,AZ,1
2,Kids Place,OH,0
3,The Toy Store,IL,1
4,Village Toys,MI,2


In ex6, we use `order by` in the query, which has different reuslt from `group by` operator.

`JOIN`

In [24]:
sql = """
        select vend_name, prod_name, prod_price
        from Vendors, Products
        where Vendors.vend_id = Products.vend_id;
        """

In [25]:
ex7 = pd.read_sql(sql, conn)
ex7

,vend_name,prod_name,prod_price
0,Bears R Us,8 inch teddy bear,5.99
1,Bears R Us,12 inch teddy bear,8.99
2,Bears R Us,18 inch teddy bear,11.99
3,Doll House Inc.,Fish bean bag toy,3.49
4,Doll House Inc.,Bird bean bag toy,3.49
5,Doll House Inc.,Rabbit bean bag toy,3.49
6,Doll House Inc.,Raggedy Ann,4.99
7,Fun and Games,King doll,9.49
8,Fun and Games,Queen doll,9.49


In [27]:
sql = """
        select vend_name, prod_name, prod_price
        from Vendors inner join Products
        on Vendors.vend_id = Products.vend_id;
        """

In [28]:
ex8 = pd.read_sql(sql, conn)
ex8

,vend_name,prod_name,prod_price
0,Bears R Us,8 inch teddy bear,5.99
1,Bears R Us,12 inch teddy bear,8.99
2,Bears R Us,18 inch teddy bear,11.99
3,Doll House Inc.,Fish bean bag toy,3.49
4,Doll House Inc.,Bird bean bag toy,3.49
5,Doll House Inc.,Rabbit bean bag toy,3.49
6,Doll House Inc.,Raggedy Ann,4.99
7,Fun and Games,King doll,9.49
8,Fun and Games,Queen doll,9.49


Multiple JOIN

In [29]:
sql = """
        select prod_name, vend_name, prod_price, quantity
        from OrderItems, Products, Vendors
        where Vendors.vend_id = Products.vend_id
        and OrderItems.prod_id = Products.prod_id
        and order_num = 20007;
        """

In [30]:
ex9 = pd.read_sql(sql, conn)
ex9

,prod_name,vend_name,prod_price,quantity
0,18 inch teddy bear,Bears R Us,11.99,50
1,Fish bean bag toy,Doll House Inc.,3.49,100
2,Bird bean bag toy,Doll House Inc.,3.49,100
3,Rabbit bean bag toy,Doll House Inc.,3.49,100
4,Raggedy Ann,Doll House Inc.,4.99,50


In [31]:
sql = """
        select cust_name, cust_contact
        from Customers, Orders, OrderItems
        where Customers.cust_id = Orders.cust_id
        and OrderItems.order_num = Orders.order_num
        and prod_id = 'RGAN01';
        """

In [32]:
ex10 = pd.read_sql(sql, conn)
ex10

,cust_name,cust_contact
0,Fun4All,Denise L. Stephens
1,The Toy Store,Kim Howard


In [33]:
sql = """
        select cust_name, cust_contact
        from Customers as C, Orders as O, OrderItems as OI
        where C.cust_id = O.cust_id
        and OI.order_num = O.order_num
        and prod_id = 'RGAN01';
        """

In [34]:
ex11 = pd.read_sql(sql, conn)
ex11

,cust_name,cust_contact
0,Fun4All,Denise L. Stephens
1,The Toy Store,Kim Howard


`self-join`

In [35]:
sql = """
        select cust_id, cust_name, cust_contact
        from Customers
        where cust_name = (select cust_name
                                     from Customers
                                     where cust_contact = 'Jim Jones');
        """

In [36]:
ex12 = pd.read_sql(sql, conn)
ex12

,cust_id,cust_name,cust_contact
0,1000000003,Fun4All,Jim Jones
1,1000000004,Fun4All,Denise L. Stephens


Using `rename` to self-join the same table

In [37]:
sql = """
        select c1.cust_id, c1.cust_name, c1.cust_contact
        from Customers as c1, Customers as c2
        where c1.cust_name = c2.cust_name
        and c2.cust_contact = 'Jim Jones';
        """

In [38]:
ex13 = pd.read_sql(sql, conn)
ex13

,cust_id,cust_name,cust_contact
0,1000000003,Fun4All,Jim Jones
1,1000000004,Fun4All,Denise L. Stephens


`natural-join`

In [39]:
sql = """
        select C.*, O.order_num, O.order_date, OI.prod_id, OI.quantity, OI.item_price
        from Customers as C, Orders as O, OrderItems as OI
        where C.cust_id = O.cust_id
        and OI.order_num = O.order_num
        and prod_id = 'RGAN01';
        """

In [40]:
ex14 = pd.read_sql(sql, conn)
ex14

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email,order_num,order_date,prod_id,quantity,item_price
0,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com,20007,2012-01-30,RGAN01,50,4.49
1,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None,20008,2012-02-03,RGAN01,5,4.99


`outer join`

In [41]:
sql = """
        select Customers.cust_id, Orders.order_num
        from Customers left outer join Orders
        on Customers.cust_id = orders.cust_id;
        """

In [42]:
ex15 = pd.read_sql(sql, conn)
ex15

,cust_id,order_num
0,1000000001,20005.0
1,1000000001,20009.0
2,1000000002,NaN
3,1000000003,20006.0
4,1000000004,20007.0
5,1000000005,20008.0


Here, `outer join` includes customers without any orders. `left` means to select all rows in the left table `Customers`.

As a comparison,

In [51]:
sql = """
        select Customers.cust_id, Orders.order_num
        from Customers inner join Orders
        on Customers.cust_id = orders.cust_id;
        """

In [52]:
pd.read_sql(sql, conn)

,cust_id,order_num
0,1000000001,20005
1,1000000003,20006
2,1000000004,20007
3,1000000005,20008
4,1000000001,20009


`join` with `aggregates`

In [66]:
sql = """
        select Customers.cust_id, count(Orders.order_num) as num_ord
        from Customers inner join Orders
        on Customers.cust_id = Orders.cust_id
        group by Customers.cust_id;
        """

In [67]:
ex16 = pd.read_sql(sql, conn)
ex16

,cust_id,num_ord
0,1000000001,2
1,1000000003,1
2,1000000004,1
3,1000000005,1


In [68]:
sql = """
        select Customers.cust_id, count(Orders.order_num) as num_ord
        from Customers left outer join Orders
        on Customers.cust_id = Orders.cust_id
        group by Customers.cust_id;
        """

In [69]:
ex17 = pd.read_sql(sql, conn)
ex17

,cust_id,num_ord
0,1000000001,2
1,1000000002,0
2,1000000003,1
3,1000000004,1
4,1000000005,1
